## install packages

In [ ]:
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz -f ./ --no-index --no-deps

In [ ]:
# install dependencies uploaded in packages folder
# !python3 -m pip install packages/ipywidgets-7.6.5.tar.gz
import subprocess
import glob
import os
from tqdm import tqdm
package_files = glob.glob(os.path.join('packages', '*.whl')) + glob.glob(os.path.join('packages', '*.tar.gz'))
# print('Installing:', package_files)
for f in tqdm(package_files, desc="Installing packages..."):
    subprocess.call(f"pip install {f} -f ./ --no-index --no-deps | tqdm", shell=True)

In [ ]:
# !pip list

In [ ]:
%cd mainfile

In [ ]:
%pwd

In [ ]:
!python dataprep.py --generate --split_ratio -1 --raw_dataset dataset/wavs_35

In [ ]:
with open('dataset/train_def.txt', 'r') as f:
    print(len(f.readlines()))
with open('dataset/val_def.txt', 'r') as f:
    print(len(f.readlines()))

# Training

### Phase 1 classifier

In [ ]:
!python main.py --do_train --augment \
                --device "cuda:2" \
                --train_list dataset/train_def.txt \
                --test_list dataset/val_def.txt \
                --model RawNet2v2 \
                --max_epoch 500 \
                --batch_size 32 \
                --nDataLoaderThread 2 \
                --criterion softmaxproto \
                --nPerSpeaker 2 \
                --n_mels 80 \
                --margin 0.2\
                --scale 30\
                --early_stop \
                --es_patience 30 \
                --weight_decay 0.000002 \
                --callbacks auto \
                --max_frames 200 \
                --initial_model exp/RawNet2v2/model/best_state_2810_9006acc.model

### Phase 2 metric loss

In [ ]:
!python main.py --do_train --augment \
                --device "cuda:2" \
                --train_list dataset/train_def.txt \
                --test_list dataset/val_def.txt \
                --save_path exp \
                --model RawNet2v2 \
                --max_epoch 500 \
                --batch_size 64 \
                --nDataLoaderThread 1 \
                --criterion softmaxproto \
                --margin 0.1\
                --scale 30\
                --nPerSpeaker 2 \
                --early_stop \
                --es_patience 30 \
                --callbacks cosinelr \
                --initial_model exp/RawNet2v2/model/best_state_2810_9006acc.model

#### E2E model

In [ ]:
!python main.py --do_train --augment \
                --train_list dataset/train_extended.txt \
                --test_list dataset/val_extended.txt \
                --model RawNet2v3 \
                --max_epoch 1000 \
                --batch_size 8 \
                --nDataLoaderThread 1 \
                --criterion softmaxproto \
                --margin 0.1\
                --scale 30\
                --nPerSpeaker 2 \
                --device "cuda:2" \
                --initial_model exp/RawNet2v3/model/best_state_smproto_320ep.model
# train lai thi de lr = 0.003 de 0.003*0.95**20 = 0.001

# Inference

In [ ]:
!python main.py --do_infer --prepare \
                --device "cuda:2" \
                --model RawNet2v2 \
                --eval_frames 100 \
                --test_list dataset/val.def.txt \
                --cohorts_path checkpoint/dump_cohorts.npy \
#                 --initial_model_infer exp/model000000500-Copy1.model

In [ ]:
%%timeit -n1 -r1
!python main.py --do_infer --eval \
                --device "cuda:2"\
                --model RawNet2v2 \
                --test_list dataset/non_noise_test.txt \
                --eval_frames 100 \
                --save_path exp \
                --cohorts_path checkpoint/dump_cohorts.npy \
                --scoring_mode cosine \
                --initial_model_infer exp/RawNet2v2/model/best_state.model
# ResNetSE34V2 RawNet2v2 RawNet2v3

### Train results

**pretrain:** <br/> 
91,08964182023294% at norm 2.4921278953552246 <br/> 
90.56% cosine <br/> 
0.8775308377099724 on noise test

**source:** <br/>
retrain and test on mainfile cosine 0.9051651474954769 , norm 90.8300321861847
0.8888843421156145 wav26

**rawnet 2v2:** <br/>
amsoftmax 89.63233473632087 0.25274425745010376, noisetest 0.8698454245440387 <br/> 
softmaxproto  0.9074772945568328(peak) noise_test 0.8830819671824<br/>
0.9064202178197283
0.9063887373796725
msloss 0.8953633751714005 <br/>

**rawnet 2v3:** <br/>
softmaxproto 0.9030544607976796 (chua train xong) noise_test 0.870850060216898 <br/>

**resnet34v2** <br/>
only amsoftmax : 0.8981813145788455 noise test 0.8726228740125874<br/>
amsoftproto 0.8960503722450445 noise test 0.8688899527926216 <br/>
only metric: 88,00139485761678 <br/>
softmaxproto: 0.9045211843177592(raw) noise-test 0.8751626266475027 <br/>

**repVGG:**
0.8371683635784459 <br/>

In [ ]:
!python main.py --do_infer --test \
                --device "cuda:2" \
                --model RawNet2v2 \
                --eval_frames 100 \
                --save_path exp \
                --cohorts_path checkpoint/cohorts_rawnet2v2.npy \
                --test_threshold 0.25274425745010376\
                --scoring_mode cosine\
                --initial_model_infer exp/RawNet2v2/model/best_state_2610_1000ep.model
#                 --initial_model_infer checkpoint/final_500.model
#                 --initial_model_infer exp/ResNetSE34V2/model/best_state_d2110_2_8561.model \


In [ ]:
import csv
#check results
def check_result(path):
    ref = "dataset/data_truth_yt_filter_noise.txt"
    com = path
    
    ref_data = {}
    com_data = {}
    

    with open(ref, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=' ')
        for row in tqdm(spamreader):
            key = f"{row[1]}/{row[-1]}"
            ref_data[key] = row[0]
            
    with open(com, newline='') as rf:
        spamreader = csv.reader(rf, delimiter=',')
        next(spamreader, None)
        for row in tqdm(spamreader):
            key = f"{row[0]}/{row[1]}"
            com_data[key] = row[2]
            
#     assert len(ref_data)==len(com_data) or len(ref_data)==(len(com_data) + 1)
    count = 0
    for k, v in com_data.items():
        if ref_data[k] == v:
            count += 1
    
    print("Precision:", count*100/len(ref_data))

check_result("exp/RawNet2v2/result/submission_list_test.csv")
    

## run deploy

In [ ]:
!python server_test.py

In [ ]:
len(glob.glob("dataset/wavs/*/*.wav")) *0.8

In [ ]:
with open('dataset/train.txt', 'r') as f:
    print(len(f.readlines()))

In [ ]:
# convert text
from pathlib import Path
test_data = []
ref = 'dataset/data_truth_yt_filter_noise.txt'
write_file = 'dataset/data_public_test.txt'

with open(ref, newline='') as rf:
    spamreader = csv.reader(rf, delimiter=' ')
    for row in tqdm(spamreader):
        test_data.append([row[0], row[1], row[-1]])

with open(write_file, 'w') as wf:
    for dt in tqdm(test_data):
        path1 = str(Path('dataset/public_test/data_test', dt[1]))
        path2 = str(Path('dataset/public_test/data_test', dt[2]))
        wf.write(f"{dt[0]} {path1} {path2}\n")


In [ ]:
import torch.nn.functional as F
import torch
import numpy as np
from numpy.linalg import norm

def cosine_simialrity(ref, com):
    similarity = torch.nn.CosineSimilarity(dim=-1, eps=1e-06)
    return np.mean(abs(similarity(ref, com)).cpu().numpy())

def cosine_sim_advance(ref, com):
    def cosine_with_adaptive(ref, com):
        com_mean = np.mean(com)
        com_cov = np.cov(com.T)
        print(com_cov)
        com_sqrt_cov = np.sqrt(com_cov)
        print(com_sqrt_cov)
        score = (np.dot((ref - com_mean).T,(com - com_mean)))/(norm(np.dot(com_sqrt_cov, ref))*norm(np.dot(com_sqrt_cov, com)))
        return score
    
    ref = ref.cpu().numpy()
    com = com.cpu().numpy()
    return np.mean(abs((cosine_with_adaptive(ref, com) + cosine_with_adaptive(com, ref))/2))
from scipy import spatial

def cosine_sim_ver2(ref, com):
#     ref = ref.cpu().numpy()
#     com = com.cpu().numpy()
    distance = torch.nn.PairwiseDistance()
    result = 1 - distance(ref, com)
    return result

In [ ]:
ref = torch.Tensor([[1.], [0.], [-1.]])
com = torch.Tensor([[-1.], [-1.], [1.]])

ref, com, cosine_sim_advance(ref, com), cosine_simialrity(ref, com), cosine_sim_ver2(ref, com)
